In [4]:
import pickle

from nnunetv2.imageio.simpleitk_reader_writer import SimpleITKIO

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import os

import SimpleITK as sitk


from nnunetv2.preprocessing.resampling.default_resampling import compute_new_shape

In [41]:
stage_1_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset104_MBAS/mbasTrainer__plans_2024_08_27__ResEncUNet_3d_lowres_for25_drop50_slim96"
stage_2_dir = "/home/bryan/expr/mbas_nnUNet_results/Dataset101_MBAS/mbasTrainer__plans_2024_08_30__ResEncUNet_p20_256_dil2_batch_dice_cascade_ResEncUNet_08_27"


data1 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug0/trainer_MBAS_005_data.npy")
logits1 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug0/trainer_MBAS_005_logits1.npy")
logits1_not_comp = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug0/trainer_not_comp_MBAS_005_logits1.npy")
logits1_not_comp2 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug0/trainer_not_comp2_MBAS_005_logits1.npy")

reader = SimpleITKIO()
seg1 = reader.read_seg(os.path.join(stage_1_dir, "crossval_results_folds_0/MBAS_005.nii.gz"))
val_original_seg1 = reader.read_seg(os.path.join(stage_1_dir, "fold_0/validation_original/MBAS_005.nii.gz"))
val_try0_seg1 = reader.read_seg(os.path.join(stage_1_dir, "fold_0/validation_try0/MBAS_005.nii.gz"))
# not compiled
val_fp32_seg1 = reader.read_seg(os.path.join(stage_1_dir, "fold_0/validation_fp32/MBAS_005.nii.gz"))
# compiled
val_seg1 = reader.read_seg(os.path.join(stage_1_dir, "fold_0/validation/MBAS_005.nii.gz"))

seg1_pp = reader.read_seg(os.path.join(stage_1_dir, "crossval_results_folds_0/postprocessed/MBAS_005.nii.gz"))
seg2 = reader.read_seg(os.path.join(stage_2_dir, "crossval_results_folds_0/MBAS_005.nii.gz"))
seg2_pp = reader.read_seg(os.path.join(stage_2_dir, "crossval_results_folds_0/postprocessed/MBAS_005.nii.gz"))

output_dir = "/home/bryan/expr/MBAS/final_submissions/output/test_debug7"
test_data1 = np.load(os.path.join(output_dir, "data1_pp/MBAS_005_label.nii.gz.npy"))
test_logits1 = np.load(os.path.join(output_dir, "logits1/MBAS_005_label.nii.gz.npy"))

test_seg1 = reader.read_seg(os.path.join(output_dir, "seg1/MBAS_005_label.nii.gz"))
test_seg1_pp = reader.read_seg(os.path.join(output_dir, "seg1_pp/MBAS_005_label.nii.gz"))
test_seg2 = reader.read_seg(os.path.join(output_dir, "seg2/MBAS_005_label.nii.gz"))
test_seg2_pp = reader.read_seg(os.path.join(output_dir, "MBAS_005_label.nii.gz"))

# test_logits1_debug4 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug4/logits1/MBAS_005_label.nii.gz.npy")

# the following 2 runs were done with torch.float32
# test_logits1_debug5 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug5/logits1/MBAS_005_label.nii.gz.npy")
# test_logits1_debug6 = np.load("/home/bryan/expr/MBAS/final_submissions/output/test_debug6/logits1/MBAS_005_label.nii.gz.npy")

In [42]:
def compare_segs(s1, s2, epsilon=1e-3):
    print(f"properties equal: {s1[1] == s2[1]}")
    array_equal = np.array_equal(s1[0], s2[0])
    print(f"array equal: {array_equal}")
    are_close = np.allclose(s1[0], s2[0], atol=epsilon)
    print(f"array allclose: {are_close}")
    
    # Find the indices where the arrays differ beyond the specified tolerance
    num_differences = np.sum(~np.isclose(s1[0], s2[0], atol=epsilon))
    print(f"array num differences: {num_differences}")

    different_indices = np.where(~np.isclose(s1[0], s2[0], atol=epsilon))
    different_tuples = list(zip(*different_indices))[:10]
#     num_differences = np.sum(s1[0] != s2[0])
#     different_indices = np.where(s1[0] != s2[0])
    print(f"different_indices: {different_tuples}")

In [43]:
compare_segs((data1, None), (test_data1, None))

properties equal: True
array equal: True
array allclose: True
array num differences: 0
different_indices: []


In [27]:
compare_segs((test_logits1, None), (test_logits1_debug4, None))

properties equal: True
array equal: False
array allclose: False
array num differences: 776384
different_indices: [(0, 0, 0, 301), (0, 0, 1, 151), (0, 0, 1, 246), (0, 0, 1, 285), (0, 0, 2, 122), (0, 0, 2, 189), (0, 0, 3, 146), (0, 0, 3, 251), (0, 0, 3, 278), (0, 0, 4, 275)]


In [28]:
compare_segs((test_logits1_debug5, None), (test_logits1_debug6, None))

properties equal: True
array equal: True
array allclose: True
array num differences: 0
different_indices: []


In [44]:
compare_segs(seg1, val_seg1)

properties equal: True
array equal: False
array allclose: False
array num differences: 5
different_indices: [(0, 9, 268, 343), (0, 20, 199, 233), (0, 20, 205, 231), (0, 22, 235, 294), (0, 25, 311, 356)]


In [32]:
compare_segs(seg1, test_seg1)

properties equal: True
array equal: False
array allclose: False
array num differences: 5
different_indices: [(0, 7, 276, 313), (0, 20, 164, 309), (0, 20, 199, 233), (0, 25, 311, 356), (0, 31, 241, 249)]


In [45]:
compare_segs(val_seg1, test_seg1)

properties equal: True
array equal: False
array allclose: False
array num differences: 1
different_indices: [(0, 22, 235, 294)]


In [40]:
# this is also not compiled
compare_segs(val_fp32_seg1, test_seg1)

properties equal: True
array equal: True
array allclose: True
array num differences: 0
different_indices: []


In [ ]:
compare_segs(seg1_pp, test_seg1_pp)

In [ ]:
compare_segs(seg2, test_seg2)

In [ ]:
compare_segs(seg2_pp, test_seg2_pp)

In [ ]:
plt.imshow(test_seg_pp[0,20], cmap="gray")

In [18]:
compute_new_shape((44, 638, 638), (2.5, 0.625, 0.625), (2.75, 0.78, 0.8))

array([ 40, 511, 498])